# PC Lab 4: Linear Classification
---

## 1. Introduction

In a binary classification setting, we are interested in assigning an observation 𝐱 to one of two possible classes, denoted by 𝑦. For example, maybe we would like to tell if a patient has a particular disease (y = 1) or not (y = 0), given certain symptoms 𝐱, in other words, we want to model $Pr(y|\mathbf{x})$, also called the *class posterior* or the *class-membership probability*. Two main approaches exist to model this probability: either directly using *discriminative models* or using *generative models*, which first model the joint probability $Pr(\mathbf{x},y)$ and obtain the class posterior using Bayes' rule.

### 1.1. Logistic regression

**Logistic regression** (LR) is the most fundamental example of a **discriminative** model. Just like linear regression, LR is a linear model. However, LR does not model the mean of a continuous outcome, but the logarithm of the [odds](https://en.wikipedia.org/wiki/Odds) of $Pr(y|\mathbf{x})$:

$$log \frac{Pr(y|\mathbf{x})}{1-Pr(y|\mathbf{x})} = w_{0}x_{0} + w_{1}x_{1} + ... + w_{p}x_{p} = \mathbf{w^Tx}.$$

In practice, we are really interested in the probability $Pr(y|\mathbf{x})$ and not in the log-odds of $Pr(y|\mathbf{x})$. To obtain this from the previous equation, we apply the *logistic function* (otherwise called a *sigmoid* function):

$$\phi(z) = \frac{1}{1 + e^{-z}} = \frac{e^{z}}{1+e^{z}}.$$

Applying this function on both sides of the LR equation, we get a model for $Pr(y|\mathbf{x})$ as follows:

$$Pr(y|\mathbf{x}) = \phi(\mathbf{w^Tx}) = \frac{1}{1 + e^{-\mathbf{w^Tx}}}.$$

If we want to classify a data point $\mathbf{x}$, we can calculate the probability of it being class $y$ with the above equation. A final classification is then made by assigning it to the class if the probability $Pr(y|\mathbf{x})$ exceeds a certain threshold, such as 0.5. 

![sigmoid](https://qph.fs.quoracdn.net/main-qimg-6ab7369356c16f17ac39fbb83d5d56c1)

### 1.2. Linear discriminant analysis

In contrast to the (discriminative) logistic regression model, linear discriminative analysis (LDA) is a **generative** model. Generative models are called as they are because they specify the class-conditional densities $Pr(\mathbf{x}|y)$, which allows to generate new observations $\mathbf{x}$ for a class $y$: for a given class $y$, we can sample from the distribution $Pr(\mathbf{x}|y)$. 

The main assumption in LDA is that the data in each class is assumed to follow a multivariate Gaussian distribution. Furthermore, the assumption is made that the (co)variance within and between predictor variables is the same for all the classes. As such, only a limited number of parameters have to be estimated from the data to fully specify the class-conditional distributions: in the multivariate case, we need to estimate the class-specific mean vectors $\mathbf{\mu_{k}}$ and the common covariance matrix $\Sigma$. In the case of just one predictor (feature), this simplifies to estimating the mean of $x$ within each class and the variance $\sigma^2$. The priors $Pr(y=k)$ are simply estimated from the data by calculating the proportion of training instances in each class.

![lda](https://d3i71xaburhd42.cloudfront.net/1ab8ea71fbef3b55b69e142897fadf43b3269463/1-Figure1-1.png)

Once the class-conditional densities $Pr(\mathbf{x}|y)$ are modeled using LDA, predictions can be obtained by using Bayes' rule:

$$Pr(y=k|\mathbf{x}) = \frac{Pr(\mathbf{x}|y=k)Pr(y=k)}{Pr(\mathbf{x})} = \frac{Pr(\mathbf{x}|y=k)Pr(y=k)}{\sum_{l=1}^{K}Pr(\mathbf{x}|y=l)Pr(y=l)},$$

where all probabilities $Pr(\mathbf{x}|y=l)$ are obtained by the LDA model and all probabilities $Pr(y=l)$ are simply obtained by means of the class frequency in the dataset (e.g. 70% of the training dataset samples belongs to class $l$, therefore, $Pr(y=l)=0.70$)

## 2. Exercises

### 2.1. Breast cancer dataset

In the following exercises, we will use the [Breast Cancer Wisconsin (Diagnostic) Data Set](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29). The dataset contains information on the disease status of 569 breast cancer patients: they were either diagnosed with a malign (status M) or with a benign (status B) tumor. 

For each patient, the dataset also contains 30 features that represent statistics of the cell nuclei present in images taken after [fine needle aspirate tissue samples](https://en.wikipedia.org/wiki/Fine-needle_aspiration). These 30 features are the mean, standard deviation and the maximum of 10 measurements on the cell nuclei:
- radius
- texture
- perimeter
- area
- smoothness
- compactness
- concavity
- concave points
- symmetry
- fractal dimension

**Based on these features of the cell nuclei, we would like to predict whether a patient has a malign or a benign breast cancer tumor.** Let's download and read in the data:

In [ ]:
!wget https://raw.githubusercontent.com/tfmortie/mlmust/main/04_linear_classification/wdbc.data

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('./wdbc.data', header=None, index_col=0, names=['Patient ID', 'status'] + list(np.arange(1,31,1)))
status = data['status']
data.head()

First, let's look at the distribution of the disease status:

In [ ]:
pd.value_counts(data['status']).plot(kind='bar');

There are about 350 benign cases and roughly 200 malign cases. This is a fairly balanced dataset.

<div class="alert alert-success">

<b>THOUGHT EXERCISE: Suppose that the dataset was unbalanced, with 525 B cases and only 25 M cases. Would you still use accuracy to evaluate the model?</b>
</div>

In these exercises, we will use the [Scikit-learn](https://scikit-learn.org/stable/) implementations of [LR](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) and [LDA](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html). 

In [ ]:
y = status
x = data.drop('status', axis=1).values # Drop the disease status from the dataframe, convert to numpy array
y[:100]

<div class="alert alert-success">

<b>EXERCISE 2.1.1: Using Scikit-learn, split the data in a 80% training and a 20% test set. Fit a logistic regression model and evaluate training and testing accuracy. </b>
</div>

Use [this method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) for train-test splitting and [this implementation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) to perform logistic regression. Accuracy can be computed by using the score() method of each model.

<div class="alert alert-success">

<b>EXERCISE 2.1.2: Now do the same with LDA. Do you split your dataset again in a new random training and testing split? Think about fair evaluation/comparison between the two models.</b>
</div>

Use [this implementation](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html) for LDA.

Depending on your data split, LDA or LR will perform better. The model that we prefer is hence determined by a random data split. One way to avoid this is to perform [cross-validation](https://machinelearningmastery.com/k-fold-cross-validation/), we will return on this topic later in the course.


<div class="alert alert-success">

<b>EXERCISE 2.1.3</b>: **Use your preferred (LDA or LR) model to predict the class probabilities $Pr(y|\mathbf{x})$ and the classes for the training data. Use the _predict_proba()_ function to generate the predicted probabilities and the _predict()_ function to generate the predicted classes. Take note of the output shape of _predict_proba()_. What do the columns mean? Use the code below to plot the two against each other. Which data points are most likely to be misclassified?**
</div>

In [ ]:
predicted_class_probabilities = "..."
predicted_classes = "..."

In [ ]:
import matplotlib.pyplot as plt

misclassified = predicted_classes !=  y_train

colors = ['#b2182b' if wrong else '#2166ac' for wrong in misclassified ]
fig, ax = plt.subplots(figsize=(8,6))
test_jitter = np.random.normal(scale=0.1, size=len(predicted_class_probabilities))
ax.scatter(test_jitter, predicted_class_probabilities,
           marker='.', s=25, color=colors, alpha=0.75)
ax.get_xaxis().set_ticks([]);
ax.set_ylabel('Predicted class probabilies').set_fontsize(20)
ax.set_title('Correctly classified samples in blue', size=24)
plt.show()

Clearly, the misclassified points are those points where the predicted probability of class membership is rather close to 0.5.

### 2.2. Multi-class classification

Some classifiers inherently support multi-class classification as part of their design, for example, in LDA, the class-conditional density $Pr(\mathbf{x}|y)$ is modeled for every class $y$ and can be converted to predictions using Bayes' rule. For logistic regression, a natural extension to the multi-class case is [multinomial logistic regression](https://en.wikipedia.org/wiki/Multinomial_logistic_regression). Check out the [Scikit-learn page on multi-class prediction](https://scikit-learn.org/stable/modules/multiclass.html) for more information. In principle, any classifier can be made multi-class using following methods:

#### One-versus-one classification

One-versus-one classification is another approach to a multi-class classification problem. For a K-class problem, the strategy consists of training $\frac{K(K-1)}{2}$ classifiers. Each of these classifiers must learn to distinguish two classes. Once the classifiers are trained, a voting scheme is applied to make a prediction for an unseen data point: each classifier has to decide between two possible classes. The final predicted class is that class that gets the largest number of votes. 

#### One-versus-all classification

In one-versus-all (OvA) classification (also called one-versus-rest), a single classifier is trained per class, with the samples of that class as positive samples and all other samples as negatives. The strategy proceeds as follows for a K-class classification problem:

**Inputs:**
* a classification algorithm L (learner)
* feature matrix $\mathbf{X}$
* label vector $\mathbf{y}$ where $y_i \in \{1,...,K\}$


**Procedure:**
for each $k$ in $\{1,...,K\}$:
* construct a new label vector $\mathbf{z}$ where $z_i$ is 1 if $y_i = k$ and 0 otherwise
* train L on $\mathbf{X}$ to obtain a classifier $f_k$. The classifier should return class probabilities and not hard labels.

**Returns:**
A list of trained classifiers $f_k$ for each $k$ in $\{1,...,K\}$

To make predictions for a new sample $\mathbf{x}$, the $k$ classifiers are applied to $\mathbf{x}$ and the final predicted label is the label that is predicted with the highest confidence (probability):

$\hat{y} = \underset{k \in \{1,...,K\}}{\mathrm{argmax}} \, f_k(\mathbf{x})$

Let's simulate a toy dataset with three classes and two features, and split it in training and test data:

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

X, y = make_blobs(n_samples=1000, centers= [[-2.5, 0], [0, 1], [3.5, -1]], random_state=42)

#train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Make the plot
fig, ax = plt.subplots(figsize=(15,10))
colors=['#66c2a5', '#fc8d62', '#8da0cb']
for i, color in enumerate(colors):
    idx_train = np.where(y_train==i)
    idx_test = np.where(y_test==i)
    plt.scatter(X_train[idx_train,0], X_train[idx_train,1], c=color, edgecolor='black', s=30)
    plt.scatter(X_test[idx_test,0], X_test[idx_test, 1],c='white', edgecolor=color, s=70)
    
ax.legend(['Class 1 - train',
           'Class 1 - test',
           'Class 2 - train',
           'Class 2 - test',
           'Class 3 - train',
           'Class 3 - test']);

ax.set_xlabel('Feature 1');
ax.set_ylabel('Feature 2');
ax.set_title('Toy dataset for multiclass classification').set_fontsize(20);

<div class="alert alert-success">
<b>EXERCISE 2.2.1: Look again at the documentation of LDA and Logistic Regression. What options do you need to specify to train these models on a multi-class problem? Train a model on this toy dataset</b>
</div>

In [ ]:
# instantiate model ...

# fit ...

predicted_classes = "..."

classification_accuracy = np.round(np.mean(y_test == predicted_classes)*100,2)

# Visualize the predictions

fig, ax = plt.subplots(figsize=(15,10))
colors=['#66c2a5', '#fc8d62', '#8da0cb']

for i, color in enumerate(colors):
    idx_train = np.where(y_train==i)
    idx_test = np.where(y_test==i)
    
    plt.scatter(X_train[idx_train,0], X_train[idx_train,1], c=color, edgecolor='black', s=30)
    plt.scatter(X_test[idx_test,0], X_test[idx_test, 1],c='white', edgecolor=color, s=70)
        
ax.legend(['Class 1 - train',
           'Class 1 - test',
           'Class 2 - train',
           'Class 2 - test',
           'Class 3 - train',
           'Class 3 - test']);

# add predictions
for i, color in enumerate(colors):
    idx_predicted = np.where(predicted_classes==i)
    plt.scatter(X_test[idx_predicted,0], X_test[idx_predicted,1], c=color, marker='s', s=2)

ax.set_xlabel('Feature 1');
ax.set_ylabel('Feature 2');
ax.set_title('Toy dataset for multiclass classification - classification accuracy: {}%'.format(classification_accuracy)).set_fontsize(20);

### 2.3. OPTIONAL: Data preprocessing for text inputs (example with DNA)

Since machine learning models expect numbers as inputs, not words, we need to do some preprocessing first to make textual data compatible with ML models. A very common feature representation for text sequences is called the **Bag of Words**. It consists of listing all the possible tokens that might occur in your text. A token might be one word, or a combination of two or multiple words. The feature representation is then a matrix with the count of the tokens for each text instance. The simplest transformation is to just look at the counts of individual tokens: the 1-grams. However, it might be interesting to count combinations of two or more words as well. Consider the following example, where we have 2 example input sample sentences:


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

data = ['this is real life',
       'is this real life']
print('data:\n', data[0], '\n', data[1])
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,2))

analyze = vectorizer.build_analyzer()
print('vocabulary:\n', analyze(data[0]))
      
features = vectorizer.fit_transform(data)
print('final feature representation:\n', features.toarray())

Note that by default, the vectorizer returns the features as a sparse array: in most applications, each row of the feature matrix will contain a lot of zeros. For this exercise, you can convert it to a regular numpy array with the ```toarray()``` method.

If we restrict ourselves to 1-grams, both sentences above are transformed into the exact same feature representation. However, suppose we want to do sentiment analysis and classify whether a sentence contains a question or not. Then it might be useful to look at bigrams as well.

In [ ]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,1))
analyze = vectorizer.build_analyzer()
analyze(data[0])
features = vectorizer.fit_transform(data)
print('Only 1-grams:')
print('final feature representation:\n', features.toarray())

When we consider bigrams, the feature representation for both sentences is no longer exactly the same. Maybe the features that represent 'this is' and 'is this' could be helpful here.

Still the bag-of-words features completely ignores where every n-gram was in the sentence. A possible way to take positional information into account is to assign $v$ dummy features for every location in the sentence, with $v$ being the vocabulary size (number of possible different words). This is called [one-hot encoding](https://en.wikipedia.org/wiki/One-hot#Natural_language_processing) and requires the sentences to be all of the same length, for example, 4 words as in the 2 sentences above.

In [ ]:
data_splitwords = [sentence.split(' ') for sentence in data]
print('data splitted by words:')
print(data_splitwords)
vocab = list(set([word for sentence in data_splitwords for word in sentence]))
print('vocabulary:')
print(vocab)
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore',
                    categories=[vocab]*len(data_splitwords[0]))

features = enc.fit_transform(data_splitwords)
print('resulting feature representation:')
print(features.toarray())

If it is not immediately clear what the features mean in this example: the first four features represent which word is the first word in each sentence. The first and sentence has as first word 'this', this is the last word in the vocabulary, so of the first four features, the last is given by $1$. The second sentence has 'is' as the first word, which is the second word in the vocabulary, for this reason, the second column of the first four features is given by $1$ for this sentence. Both sentences have the same last two words 'real life', it can be seen that the last 8 features for both sentences are identical.

Armed with these encoding schemes, we can tackle a real problem. **DNA sequences** as inputs are peculiar because they are essentially text: following an alphabet containing only A, T, C or G. We will use data from a published [study](https://bmcbiol.biomedcentral.com/articles/10.1186/1741-7007-12-4) about sigma factors in E. coli. Some biological background information for those interested:

> The binding region of the transcription unit,  commonly called the promoter region, is known to play a key role  in the transcription rate of downstream genes. In Eubacteria, the sigma factor ($\sigma$) binds with the RNA  polymerase subunit ($\alpha\beta\beta'\omega$) to create RNA polymerase. Being part of the RNA polymerase holoenzyme, the sigma element acts as the connection between RNA polymerase and DNA. Depending on the sigma factor bound to the transcription unit, different binding regions of the enzyme with the DNA are observed. The data represents specific DNA regions of the *E. coli* genome to which one or more sigma factors bind.

In less-biological-jargon words: certain molecules called sigma factors control activity of genes, they do so by binding to the DNA close to the gene. If we can predict for an input DNA sequence if such a molecule will bind, we gain an understanding in the biological function of those genes and molecules.

The dataset contains 4724 DNA sequences. For each sequence, we have information on five target variables, each representing 5 sigma factors. Each time, the outcome is binary, indicating whether the sigma factor binds to the sequence or not. This is a **multi-label** problem, where an input can belong to any (multiple at once) of the classes, as opposed to only one possible class for each sample in multi-class. For simplicity in this exercise, we will focus only on one class: RPOD.

In [ ]:
!wget https://raw.githubusercontent.com/tfmortie/mlmust/main/04_linear_classification/promotor_list_exp_growth.csv
promotor_data = pd.read_csv('./promotor_list_exp_growth.csv', index_col=0)
promotor_data.head(20)

X = np.array(promotor_data.PROBE_SEQUENCE)
y = promotor_data['RPOD'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=23)

<div class="alert alert-success">

<b> EXERCISE: 
Use one of the encoding schemes to preprocess the DNA data. Construct a model and evaluate performance on the test set. Play around with different preprocessing values, and keep testing performances. What types of preprocessing lead to good performance for this dataset? In what cases are we overfitting? Think about fair evaluation: if we keep testing out configurations and evaluating on the test set, is our final test performance estimate fair? We can solve this by keeping an additional data split separate: training, validation and testing. Then we test different hyperparameters (different ways of preprocessing) on the validation set. We don't touch the test set until we have our final model. This is the only fair way of evaluating. For more information, see this [link](https://en.wikipedia.org/wiki/Training,_validation,_and_test_sets).</b>
</div></b>
</div>

In [ ]:
# instantiate encoder

# fit and transform data with encoder

# instantiate modle

# fit model

# evaluate